# Homework 2: Word Similarity

Student Name: Minh Trung Nguyen

Student ID: 1016151

## General info

<b>Due date</b>: Thursday, 4 June 2020 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 10% of mark for class (with 9% on correctness + 1% on quality and efficiency of your code)

**Note**: As we will be implementing neural networks in this assignment, you're encouraged to build your notebook on **colab**. See the programming exercise in workshop-07 (`10-bert.ipynb`) if you are not familiar with colab.

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/17601/pages/using-jupyter-notebook-and-python?module_item_id=1678430) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. Deep learning libraries such as keras and pytorch are also allowed.  You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.




<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dataset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

### Question 1 (1.0 mark)

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://alfonseca.org/eng/research/wordsim353.html">Similarity-353</a>. You need to first obtain this dataset, which is on Canvas (assignment 2). The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on **document frequencies** (not token frequencies) in the Brown corpus, in order to remove rare words. In this homework, **we will be treating the paragraphs of the Brown corpus as our "documents"**. You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized (lemmatize function provided). Store this preprocessed data in *brown_corpus* object (it will be used for question 4 and 5 later).

Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of **$< 8$** in this corpus. You should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*.

Note: the document frequency of a word denotes the number of documents that contains the word.

**Task**: Filter word pairs from *set1.tab* based on document frequencies. Produce *brown_corpus*, which is a list where each element is a set of words for one paragraph (e.g. the first element in *brown_corpus* should contain all the unique word types for the first paragraph). Produce *filtered_gold_standard*, a dictionary of filtered word pairs with human similarity ratings (the dictionary should have (word1, word2) as keys, and similarity ratings as values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *filtered_gold_standard*.

In [1]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

# lemmatizer
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


###
# Your answer BEGINS HERE
###

# create brown corpus
brown_corpus = []
for document in brown.paras():
    temp = []
    for i in range(len(document)):
        for token in document[i]:
            if token.isalpha():
                temp.append(lemmatize(token.lower()))
    brown_corpus.append(list(set(temp)))

# loading the file to python dictionary  
data = {}
with open("set1.tab") as f:
    rows = (line.split('\t') for line in f)
    for row in rows:
        if row[0] == "Word 1":
            continue
        else: 
            data[(row[0], row[1])] = float(row[2])
    

# calculate document frequency for word types and store it in a dict
doc_freq = {} # the length of doc_freq is vocab_size
for para in brown_corpus:
    for token in para:
        if token not in doc_freq:
            doc_freq[token] = 1
        else:
            doc_freq[token] += 1

# filtering the rare words
for word_pairs in data:
    if (word_pairs[0] not in doc_freq) or (word_pairs[1] not in doc_freq):
        continue
    elif (doc_freq[word_pairs[0]]) < 8 or (doc_freq[word_pairs[1]] < 8):
        continue
    else:
        filtered_gold_standard[word_pairs] = data[word_pairs]            
            
###
# Your answer ENDS HERE
###

print(len(filtered_gold_standard))
print(filtered_gold_standard)

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\nghet\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nghet\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


94
{('love', 'sex'): 6.77, ('tiger', 'cat'): 7.35, ('tiger', 'tiger'): 10.0, ('book', 'paper'): 7.46, ('plane', 'car'): 5.77, ('train', 'car'): 6.31, ('telephone', 'communication'): 7.5, ('television', 'radio'): 6.77, ('drug', 'abuse'): 6.85, ('bread', 'butter'): 6.19, ('doctor', 'nurse'): 7.0, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('smart', 'student'): 4.62, ('smart', 'stupid'): 5.81, ('company', 'stock'): 7.08, ('stock', 'market'): 8.08, ('stock', 'phone'): 1.62, ('stock', 'egg'): 1.81, ('stock', 'live'): 3.73, ('stock', 'life'): 0.92, ('book', 'library'): 7.46, ('bank', 'money'): 8.12, ('wood', 'forest'): 7.73, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('holy', 'sex'): 1.62, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('tennis', 'racket'): 7.56, ('law', 'lawyer'): 8.38, ('movie', 'star'): 7.38, ('movie', 'critic'): 6.73, ('movie', 'theater'): 7.92, ('space', 'chemistry'): 4.88, ('alcohol', 'chemistry'): 

<b>For your testing: </b>

In [2]:
assert(len(brown_corpus)==15667)
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

### Question 2 (1.0 mark)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a **single primary sense**. We define single primary sense here as either: (a) having only one sense (i.e. only one synset), or (b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma.

Note: You should lowercase the lemmas of a synset when matching your word; and if there are multiple lowercased lemmas that match your word, you should sum up the count of all matching lemmas.

Additionally, you should remove any words where the primary sense is **not a noun** (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section.

Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria. When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*.

**Task**: Filter word pairs for any words which do not have a single primary sense and aren't nouns. Produce *final_gold_standard*, a dictionary of filtered word pairs with human similarity ratings. Note: this second filtering is applied on top of the first filtering (question 1). In other words, you shouldn't consider any word pairs that have already been discarded by the first filtering in this question.

**Check**: Use the assertion statements in *"For your testing"* for the expected *final_gold_standard*.

In [3]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}
word_primarysense = {} #a dictionary of (word, primary_sense) (used for next section); primary_sense is a synset

###
# Your answer BEGINS HERE
###
from collections import Counter

# check if the primary sense of a word is a noun or not
def is_noun(synset):
    if synset.name().split(".")[1] == "n":
        return True
    else:
        return False

# function to check if a word has a primary sense, if yes, return True
def is_primary(word):
    if len(wordnet.synsets(word)) == 1:
        if is_noun(wordnet.synsets(word)[0]):
            word_primarysense[word] = wordnet.synsets(word)[0]
            return True
        else:
            return False
    else:
        counts = Counter()
        synsets = wordnet.synsets(word)
        for synset in synsets:
            lemmas = synset.lemmas()
            for lemma in lemmas:
                if lemma.name().lower() == word:
                    if synset not in counts:
                        counts[synset] = counts.get(synset, 0) + lemma.count()
                    else:
                        counts[synset] = counts.get(synset) + lemma.count()
        
        counter = counts.most_common() # return a list of ordered tuple (synset, count)
        most_common_synset = counter[0][0]
        second_common_synset = counter[1][0]
       
        if counts[second_common_synset] > 0:
            if (counts[most_common_synset] / counts[second_common_synset] >= 4) and is_noun(most_common_synset):
                if is_noun(most_common_synset):
                    word_primarysense[word] = most_common_synset
                    return True
            else:
                return False
        else:
            # if word has 2nd most common sense with count = 0 but the most common has count > 0, return True
            if counts[most_common_synset] > 0:
                if is_noun(most_common_synset):
                    word_primarysense[word] = most_common_synset
                    return True
            return False

# Finally, filter the final set
for word_pairs in filtered_gold_standard:
    if (is_primary(word_pairs[0]) == True) and (is_primary(word_pairs[1]) == True):
        final_gold_standard[word_pairs] = filtered_gold_standard[word_pairs]
        

###
# Your answer ENDS HERE
###

print(len(final_gold_standard))
print(final_gold_standard)

26
{('bread', 'butter'): 6.19, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('stock', 'egg'): 1.81, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('alcohol', 'chemistry'): 5.54, ('baby', 'mother'): 7.85, ('car', 'automobile'): 8.94, ('journey', 'voyage'): 9.29, ('coast', 'shore'): 9.1, ('brother', 'monk'): 6.27, ('journey', 'car'): 5.85, ('coast', 'hill'): 4.38, ('forest', 'graveyard'): 1.85, ('monk', 'slave'): 0.92, ('coast', 'forest'): 3.15, ('psychology', 'doctor'): 6.42, ('psychology', 'mind'): 7.69, ('psychology', 'health'): 7.23, ('psychology', 'science'): 6.71, ('planet', 'moon'): 8.08, ('planet', 'galaxy'): 8.11}


In [4]:
word_primarysense

{'book': Synset('book.n.01'),
 'television': Synset('television.n.01'),
 'drug': Synset('drug.n.01'),
 'bread': Synset('bread.n.01'),
 'butter': Synset('butter.n.01'),
 'doctor': Synset('doctor.n.01'),
 'professor': Synset('professor.n.01'),
 'student': Synset('student.n.01'),
 'stock': Synset('stock.n.01'),
 'egg': Synset('egg.n.01'),
 'money': Synset('money.n.01'),
 'cash': Synset('cash.n.01'),
 'king': Synset('king.n.01'),
 'queen': Synset('queen.n.01'),
 'bishop': Synset('bishop.n.01'),
 'rabbi': Synset('rabbi.n.01'),
 'football': Synset('football.n.01'),
 'basketball': Synset('basketball.n.01'),
 'tennis': Synset('tennis.n.01'),
 'movie': Synset('movie.n.01'),
 'alcohol': Synset('alcohol.n.01'),
 'chemistry': Synset('chemistry.n.01'),
 'baby': Synset('baby.n.01'),
 'mother': Synset('mother.n.01'),
 'car': Synset('car.n.01'),
 'automobile': Synset('car.n.01'),
 'journey': Synset('journey.n.01'),
 'voyage': Synset('ocean_trip.n.01'),
 'coast': Synset('seashore.n.01'),
 'shore': Syns

<b>For your testing:</b>

In [5]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Computing word similiarity with Lin similarity, NPMI and LSA (3 marks)

### Question 3 (1.0 mark)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*.

**Task**: Compute word pair similarity using Lin similarity for the test set. Produce *lin_similarities*, a dictionary of word pairs (keys) and computed Lin similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *lin_similarities*. 

In [6]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###
brown_ic = wordnet_ic.ic('ic-brown.dat')
for word_pair in final_gold_standard:
    lin_sim = word_primarysense[word_pair[0]].lin_similarity(word_primarysense[word_pair[1]], brown_ic)
    lin_similarities[word_pair] = lin_sim
###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     C:\Users\nghet\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


{('bread', 'butter'): 0.711420490146294, ('professor', 'doctor'): 0.7036526610448273, ('student', 'professor'): 0.26208607023317687, ('stock', 'egg'): -0.0, ('money', 'cash'): 0.7888839126424345, ('king', 'queen'): 0.25872135992145145, ('bishop', 'rabbi'): 0.6655650900427844, ('football', 'basketball'): 0.7536025025710653, ('football', 'tennis'): 0.7699955045932811, ('alcohol', 'chemistry'): 0.062235427146896456, ('baby', 'mother'): 0.6315913189894092, ('car', 'automobile'): 1.0, ('journey', 'voyage'): 0.6969176573027711, ('coast', 'shore'): 0.9632173804623256, ('brother', 'monk'): 0.24862817480738675, ('journey', 'car'): -0.0, ('coast', 'hill'): 0.5991131628821826, ('forest', 'graveyard'): -0.0, ('monk', 'slave'): 0.2543108201944307, ('coast', 'forest'): -0.0, ('psychology', 'doctor'): -0.0, ('psychology', 'mind'): 0.304017384194818, ('psychology', 'health'): 0.06004979886905243, ('psychology', 'science'): 0.8474590505736942, ('planet', 'moon'): 0.7808882364067532, ('planet', 'galaxy'

<b>For your testing:</b>

In [7]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

### Question 4 (1.0 mark)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-log_2(p(x,y))} = \frac{log_2(p(x)p(y))}{log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NPMI is -1. NPMI is normalized between [-1, +1]. 

You should use the *brown_corpus* object you've created in question 1 as your corpus here. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*.

**Task**: Compute word pair similarity using NPMI similarity for the test set. Produce *NPMI_similarities*, a dictionary of word pairs (keys) and computed NPMI similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *NPMI_similarities*.

In [8]:
n_word_types = 0
for para in brown_corpus:
    n_word_types += len(para)
print(n_word_types)

672602


In [9]:
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

###
# Your answer BEGINS HERE
###
import math

# NPMI function
def NPMI_similarity(word_pair):
    NPMI = 0
    px = doc_freq[word_pair[0]] / n_word_types
    py = doc_freq[word_pair[1]] / n_word_types
    # calculate number of co-occur
    n_co_occur = 0
    for para in brown_corpus:
        if (word_pair[0] in para) and (word_pair[1] in para): 
            n_co_occur += 1 
    pxy = n_co_occur / n_word_types
    if pxy == 0: # no co-occurence
        return -1
    PMI = math.log2(pxy / (px*py))
    NPMI = PMI / (-math.log2(pxy))
    return NPMI

# calculate for the test set
for word_pair in final_gold_standard:
    NPMI_sim = NPMI_similarity(word_pair)
    NPMI_similarities[word_pair] = NPMI_sim

###
# Your answer ENDS HERE
###

print(NPMI_similarities)

{('bread', 'butter'): 0.6531272737497535, ('professor', 'doctor'): -1, ('student', 'professor'): 0.5354959954009257, ('stock', 'egg'): 0.36851289222137446, ('money', 'cash'): 0.4449383472351726, ('king', 'queen'): 0.41814072971395816, ('bishop', 'rabbi'): -1, ('football', 'basketball'): 0.7161994042283006, ('football', 'tennis'): -1, ('alcohol', 'chemistry'): 0.6246376972254173, ('baby', 'mother'): 0.5149353890388502, ('car', 'automobile'): 0.5430334549802616, ('journey', 'voyage'): -1, ('coast', 'shore'): 0.5861510629843373, ('brother', 'monk'): 0.42993185256931743, ('journey', 'car'): -1, ('coast', 'hill'): 0.33931028834846233, ('forest', 'graveyard'): -1, ('monk', 'slave'): -1, ('coast', 'forest'): 0.45787396913892875, ('psychology', 'doctor'): 0.4613281583726411, ('psychology', 'mind'): 0.44616660434351807, ('psychology', 'health'): -1, ('psychology', 'science'): 0.5908740096534866, ('planet', 'moon'): 0.6580006162323319, ('planet', 'galaxy'): -1}


<b>For your testing:</b>

In [10]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

### Question 5 (1.0 mark)

**Instructions:** Here we'll use singular value decomposition (SVD) to derive similarity scores using the Latent Semantic Analysis (LSA) method. Recall that LSA applies SVD and truncation to get a dense vector representation of a word type. To measure similarity between two words we calculate cosine similarity between the LSA vectors of the words.

We'll first build a term-document frequency matrix (as before, a document is a paragraph in Brown corpus). That is, the rows corresponds to words in the vocabulary, and the columns represent the paragraphs/documents in Brown corpus. Each cell records the document frequency of a word (1 if the word appears in the document, 0 otherwise). You should use the *brown_corpus* object created in question 1 as your corpus to build the term-document frequency matrix.

Given the term-document frequency matrix, we'll use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for the word pairs in the test set.

You can use the workshop's notebook on distributional similarity (``09-distributional-similarity.ipynb``) as a starting point, but note that we use term-document frequency matrix (as opposed to the tf-idf matrix in the workshop).

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*.

**Task**: Compute word pair similarity using LSA. Produce *LSA_similarities*, a dictionary of word pairs (keys) and computed LSA similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *LSA_similarities*. 



In [11]:
from sklearn.feature_extraction.text import CountVectorizer

brown_corpus_join = [" ".join(para) for para in brown_corpus] # need to join the tokens to use as input to the CountVectorizer
vectorizer = CountVectorizer()
brown_matrix = vectorizer.fit_transform(brown_corpus_join)
print(brown_matrix)

  (0, 17482)	1
  (0, 19487)	1
  (0, 20220)	1
  (0, 17898)	1
  (0, 21067)	1
  (0, 928)	1
  (0, 20142)	1
  (0, 10195)	1
  (0, 13966)	1
  (0, 10907)	1
  (0, 13465)	1
  (0, 5848)	1
  (0, 25752)	1
  (0, 8806)	1
  (0, 13556)	1
  (0, 8144)	1
  (0, 10103)	1
  (0, 25393)	1
  (0, 1161)	1
  (0, 22538)	1
  (0, 25761)	1
  (1, 17898)	1
  (1, 13966)	1
  (1, 25752)	1
  (1, 8144)	1
  :	:
  (15666, 23672)	1
  (15666, 3579)	1
  (15666, 16539)	1
  (15666, 15220)	1
  (15666, 6832)	1
  (15666, 19307)	1
  (15666, 25233)	1
  (15666, 10659)	1
  (15666, 604)	1
  (15666, 10999)	1
  (15666, 4408)	1
  (15666, 26495)	1
  (15666, 3978)	1
  (15666, 2533)	1
  (15666, 23345)	1
  (15666, 14922)	1
  (15666, 10415)	1
  (15666, 16330)	1
  (15666, 7540)	1
  (15666, 28094)	1
  (15666, 8897)	1
  (15666, 4379)	1
  (15666, 1833)	1
  (15666, 3069)	1
  (15666, 24800)	1


In [12]:
brown_matrix.shape # num_documents x vocab_size 

(15667, 29104)

In [13]:
# Using SVD to create dense vector of length k=500 for the term-document matrix
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

brown_matrix_transposed = csr_matrix(brown_matrix).transpose()

svd = TruncatedSVD(n_components=500)
brown_matrix_lowrank = svd.fit_transform(brown_matrix_transposed)

print(brown_matrix_lowrank.shape)
print(brown_matrix_lowrank)

(29104, 500)
[[ 8.72928647e-03  1.04073872e-02  5.23869165e-03 ...  4.65676816e-03
  -4.30215753e-03 -2.06589140e-03]
 [ 7.08687305e-03 -7.27719615e-03  1.46339331e-03 ...  2.23434459e-03
  -5.18105888e-04  4.38847622e-04]
 [ 5.51438072e-03 -2.56928266e-03  1.26191854e-02 ...  2.43601781e-03
  -2.12369015e-05  4.22623629e-03]
 ...
 [ 1.05420117e-02 -1.53914677e-02  8.97164880e-03 ...  8.09394548e-03
   3.47996337e-03  5.00830520e-03]
 [ 1.59458259e-02 -3.70873524e-05 -2.31467871e-02 ...  2.58823125e-02
  -7.90197701e-03 -1.82384918e-02]
 [ 1.75701798e-02 -2.23879303e-02 -3.64418448e-04 ...  1.79475162e-03
  -9.60951203e-03 -2.23141479e-02]]


In [14]:
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}


import numpy as np
from numpy.linalg import norm

def cos_sim(a, b):
    return np.dot(a, b)/(norm(a)*norm(b))

# calculate the test set 
for word_pair in final_gold_standard:
    v1 = brown_matrix_lowrank[vectorizer.vocabulary_[word_pair[0]]]
    v2 = brown_matrix_lowrank[vectorizer.vocabulary_[word_pair[1]]]
    LSA_similarities[word_pair] = cos_sim(v1, v2)


print(LSA_similarities)

{('bread', 'butter'): 0.3053816976433358, ('professor', 'doctor'): 0.054175557732328294, ('student', 'professor'): 0.2938265083423611, ('stock', 'egg'): 0.08047328119162034, ('money', 'cash'): 0.12962079385310396, ('king', 'queen'): 0.0980956647980003, ('bishop', 'rabbi'): 0.020604552799466224, ('football', 'basketball'): 0.2829249792416814, ('football', 'tennis'): 0.12360250691167989, ('alcohol', 'chemistry'): 0.09222131914849037, ('baby', 'mother'): 0.3753354446953047, ('car', 'automobile'): 0.3599676613401671, ('journey', 'voyage'): 0.0811736632034695, ('coast', 'shore'): 0.42327212975998635, ('brother', 'monk'): 0.10409782091453262, ('journey', 'car'): 0.00938199001459409, ('coast', 'hill'): 0.22141408117995107, ('forest', 'graveyard'): 0.05042233272846521, ('monk', 'slave'): 0.013325183121620988, ('coast', 'forest'): 0.18366034472540135, ('psychology', 'doctor'): 0.17301443128654906, ('psychology', 'mind'): 0.12138818985483919, ('psychology', 'health'): 0.08408824704791935, ('psyc

<b>For your testing:</b>

In [15]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## Computing word similarity with feedforward language model (3 marks)

### Question 6 (1.0 mark)

**Instructions**: Here we'll build a n-gram neural language model to learn word embeddings, and compute word similarity based on the word embeddings. As before we will use the Brown corpus as training data for our language model, and the first step is to collect the vocabulary.

As before, we'll treat paragraphs in the Brown corpus as our documents. The first 12K paragraphs/documents will serve as our training data, and the rest (3K+ documents) as development data. The first step towards building a language model is to collect the vocabulary, i.e. the set of unique word types in our training data. When collecting the word types, you should lowercase all words, and only keep word types that have a frequency $>= 5$. Store your vocabulary in the _vocab_ object.

Note: 
  - we'll be using _$<$UNK$>$_ to represent unseen words, and so _vocab_ is initialised with the special _$<$UNK$>$_.
  - you do not need to do any other additional preprocessing aside from the aforementioned steps (e.g. no need to remove symbols, etc)
  - you should not use *brown_corpus* here, as you need the words in their original order (*brown_corpus* stores a set of words for each paragraph, and so do not contain word order information)

**Task**: Collect a set of unique word types in the training portion (first 12K paragraphs) of the Brown corpus. Produce _vocab_, which is a set that contains all the word types.

**Check**: Use the assertion statements in *"For your testing"* below to check the vocabulary size.

In [16]:
# create brown corpus for this task
brown_corpus_NN = []
for document in brown.paras():
    para = []
    for i in range(len(document)):
        sent = []
        for token in document[i]:
            sent.append(token.lower())
        para.append(sent)
    brown_corpus_NN.append(para)

In [17]:
training_set = brown_corpus_NN[:12000]
dev_set = brown_corpus_NN[12000:]

In [18]:
# calculate term frequency for word types and store it in a dict
doc_freq_NN = {} 
for para in training_set:
    for i in range(len(para)):
        for token in para[i]:
            if token not in doc_freq_NN:
                doc_freq_NN[token] = 1
            else:
                doc_freq_NN[token] += 1

In [19]:
len(doc_freq_NN)

45413

In [20]:
num_train = 12000
UNK_symbol = "<UNK>"
vocab = set([UNK_symbol])


# only add word with frequency >= 5 to the vocab
for word in doc_freq_NN:
    if doc_freq_NN[word] >= 5:
        vocab.add(word)


print(len(vocab))

12681


In [21]:
vocab

{'r',
 'reluctance',
 'refusing',
 'induced',
 'openings',
 'typical',
 'indispensable',
 'columbus',
 'ice',
 'uneasy',
 'rare',
 'power',
 'appealing',
 'attempt',
 'them',
 'ours',
 'builds',
 'accidental',
 'tangents',
 'arrived',
 'impact',
 'desk',
 'banner',
 'swinging',
 'official',
 'ludie',
 'amy',
 'further',
 'unhappily',
 'schedules',
 'responsibilities',
 'ride',
 'conversion',
 'stranger',
 'rails',
 'capitol',
 'belong',
 'contend',
 'bushes',
 'obedience',
 'sped',
 'pile',
 'beckett',
 'kohnstamm-positive',
 'expanding',
 'strokes',
 'promote',
 'exert',
 'cousins',
 'quite',
 'encourage',
 'muttered',
 'periodically',
 'downward',
 'imperative',
 'hours',
 'disrupt',
 'pseudophloem',
 'pip',
 'complained',
 'recall',
 'wildly',
 'rolls',
 'tolerance',
 'knight',
 'educators',
 'publishing',
 'timed',
 'yield',
 'larry',
 'bits',
 'posse',
 'joke',
 'modern',
 'ingenious',
 'jo',
 'michelangelo',
 'tap',
 'slowed',
 'unpleasant',
 'because',
 'disturbance',
 'decorati

**For your testing:**

In [22]:
assert(len(vocab) > 8000 and len(vocab) < 20000)

### Question 7 (1.0 mark)

**Instructions**: As we'll be building a trigram neural language model (based on lecture 7, page 20), the next step is to collect trigrams to construct our training data. In a trigram neural language model, for example if we have the trigram _cow eats grass_, the input to the model is the first two terms of a trigram (_cow_ and _eats_), and the language model's aim is to predict the last term of the trigram (_grass_). Your task here is to construct the training and development data for the language model. Just like the previous step, the first 12K paragraphs will serve as our training data, and the remaining 3K+ will be for development. You'll need to map words into IDs when constructing the training and development data. Any words that are not in _vocab_ should be mapped to the special _$<$UNK$>$_ symbol.

As an example, given the sentence "_a big fat hungry cow ._", you should create the following training examples:

|input|target|
|:---:|:----:|
|<i>a</i>, _big_|_fat_|
|_big_, _fat_|_hungry_|
|_fat_, _hungry_|_cow_|
|_hungry_, _cow_|_._|


Note:
   - _vocab_ is a set and so does not map words into IDs. You'll need to create a word to ID mapping first based on _vocab_.
   - A trigram should not cross sentence boundary.
   - You should ignore sentences that have less than 3 words (as they are too short to form trigrams).
   - We won't need special starting symbol when collecting the trigrams, as we are only interested in learning word embeddings here (and not computing probabilities of a sentence).


**Task**: Create training and development data. The training input and target data should be stored in the <i>x_train</i> and <i>y_train</i> respectively, and development input and target data in <i>x_dev</i> and <i>y_dev</i> respectively.

**Check**: Use the assertion statements in *"For your testing"* below for the expected shape of the outputs.

In [23]:
# Word to ID mapping
ID = 1
word_IDs = {}
for word in vocab:
    word_IDs[word] = ID
    ID += 1

In [24]:
def create_trigrams(data):
    trigrams = []
    for para in data:
        for i in range(len(para)):
            if len(para[i]) < 3:
                continue
            for j in range(0, len(para[i]) - 2):
                trigrams.append((para[i][j], para[i][j+1], para[i][j+2]))
    return trigrams

trigrams_train = create_trigrams(training_set)
trigrams_dev = create_trigrams(dev_set)

In [25]:
trigrams_train

[('the', 'fulton', 'county'),
 ('fulton', 'county', 'grand'),
 ('county', 'grand', 'jury'),
 ('grand', 'jury', 'said'),
 ('jury', 'said', 'friday'),
 ('said', 'friday', 'an'),
 ('friday', 'an', 'investigation'),
 ('an', 'investigation', 'of'),
 ('investigation', 'of', "atlanta's"),
 ('of', "atlanta's", 'recent'),
 ("atlanta's", 'recent', 'primary'),
 ('recent', 'primary', 'election'),
 ('primary', 'election', 'produced'),
 ('election', 'produced', '``'),
 ('produced', '``', 'no'),
 ('``', 'no', 'evidence'),
 ('no', 'evidence', "''"),
 ('evidence', "''", 'that'),
 ("''", 'that', 'any'),
 ('that', 'any', 'irregularities'),
 ('any', 'irregularities', 'took'),
 ('irregularities', 'took', 'place'),
 ('took', 'place', '.'),
 ('the', 'jury', 'further'),
 ('jury', 'further', 'said'),
 ('further', 'said', 'in'),
 ('said', 'in', 'term-end'),
 ('in', 'term-end', 'presentments'),
 ('term-end', 'presentments', 'that'),
 ('presentments', 'that', 'the'),
 ('that', 'the', 'city'),
 ('the', 'city', 'ex

In [26]:
len(trigrams_train)

872823

In [27]:
# create X_train and y_train
x_train = []
y_train = []
for trigram in trigrams_train:
    temp = []
    if trigram[0] in vocab:
        temp.append(word_IDs[trigram[0]])
    else:
        temp.append(word_IDs["<UNK>"])
    if trigram[1] in vocab:
        temp.append(word_IDs[trigram[1]])
    else:
        temp.append(word_IDs["<UNK>"])
    x_train.append(temp)
    
    if trigram[2] in vocab:
        y_train.append(word_IDs[trigram[2]])
    else:
        y_train.append(word_IDs["<UNK>"])
x_train = np.array(x_train)
y_train = np.array(y_train)

In [28]:
# create x_dev and y_dev
x_dev = []
y_dev = []
for trigram in trigrams_dev:
    temp = []
    if trigram[0] in vocab:
        temp.append(word_IDs[trigram[0]])
    else:
        temp.append(word_IDs["<UNK>"])
    if trigram[1] in vocab:
        temp.append(word_IDs[trigram[1]])
    else:
        temp.append(word_IDs["<UNK>"])
    x_dev.append(temp)
    
    if trigram[2] in vocab:
        y_dev.append(word_IDs[trigram[2]])
    else:
        y_dev.append(word_IDs["<UNK>"])
x_dev = np.array(x_dev)
y_dev = np.array(y_dev)

In [29]:
print(x_train.shape)
print(y_train.shape)
print(x_dev.shape)
print(y_dev.shape)

(872823, 2)
(872823,)
(174016, 2)
(174016,)


**For your testing:**

In [30]:
assert(x_train.shape[0] == y_train.shape[0])
assert(x_dev.shape[0] == y_dev.shape[0])
assert(x_train.shape[0] > 500000)
assert(x_dev.shape[0] > 50000)

### Question 8 (1.0 mark)

**Instructions**: Now let's build the trigram neural language model. We'll use the language model described in lecture 7 (page 20):

$x' = e(x_1) \oplus e(x_2)$

$h = \tanh(W_1 x' + b)$

$y = $ softmax$(W_2 h)$

where $\oplus$ is the concatenation operation, $x_1$ and $x_2$ are the input words, $e$ is an embedding function, and $y$ is the target word. You can use either `keras` or `pytorch` for building your model.

Set the dimension of the word embeddings and $h$ to 100, and train your model with 3 epochs with a batch size of 256. You will not need to tune hyper-parameters for this task.

After the model is trained, use the word embeddings to compute word similarity in the test set. Store the similarity scores in a dictionary called *lm_similarities*.

Note:
  - For words in the test set that are not in your vocabulary, you should treat them as unknown words. In other words, you should use $<$UNK$>$'s embedding to represent these words.
  - The training may take some time on CPU. You can run your notebook on colab with a GPU if you want faster training (see the programming exercise in workshop-07 if you're not familiar with colab)

**Task**: Train a trigram neural language model, and use the learned word embeddings to compute word similarity. Produce *lm_similarities*, a dictionary that contains word pairs as keys and similarity scores as values.

In [31]:
import numpy as np
embedding_size = 100
vocab_size = len(vocab)
batch_size=256

In [32]:
vocab_size

12681

In [33]:
from keras.models import Sequential
from keras import layers
from keras import losses

# word order preserved with this architecture
model2 = Sequential(name="feedforward-sequence-input")
model2.add(layers.Embedding(input_dim=vocab_size+1, 
                           output_dim=embedding_size, 
                           input_length=2))
model2.add(layers.Flatten())
model2.add(layers.Dense(100, activation='tanh'))
model2.add(layers.Dense(vocab_size+1, activation='softmax'))
model2.compile(optimizer='adam',
              loss=losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])
model2.summary()

Using TensorFlow backend.


Model: "feedforward-sequence-input"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 100)            1268200   
_________________________________________________________________
flatten_1 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_2 (Dense)              (None, 12682)             1280882   
Total params: 2,569,182
Trainable params: 2,569,182
Non-trainable params: 0
_________________________________________________________________


In [34]:
# training
model2.fit(x_train, y_train, epochs=3, verbose=True, validation_data=(x_dev, y_dev), batch_size=batch_size)

c:\users\nghet\documents\nlp project\project-files\venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 872823 samples, validate on 174016 samples
Epoch 1/3
872823/872823 [==============================] - 605s 693us/step - loss: 5.9356 - accuracy: 0.1501 - val_loss: 5.2548 - val_accuracy: 0.1661
Epoch 2/3
872823/872823 [==============================] - 561s 643us/step - loss: 5.4324 - accuracy: 0.1838 - val_loss: 5.1312 - val_accuracy: 0.1718
Epoch 3/3
872823/872823 [==============================] - 525s 601us/step - loss: 5.2310 - accuracy: 0.1954 - val_loss: 5.0878 - val_accuracy: 0.1752


In [35]:
lm_similarities = {}

###
# Your answer BEGINS HERE
###

embeddings = model2.get_layer(index=0).get_weights()[0] #word embeddings

for word_pair in final_gold_standard:
    if word_pair[0] in vocab:
        v1 = embeddings[word_IDs[word_pair[0]]-1]
    else:
        v1 = embeddings[word_IDs["<UNK>"]-1]
    if word_pair[1] in vocab:
        v2 = embeddings[word_IDs[word_pair[1]]-1]
    else:
        v2 = embeddings[word_IDs["<UNK>"]-1]
    lm_similarities[word_pair] = cos_sim(v1, v2)

###
# Your answer ENDS HERE
###


print(lm_similarities)

{('bread', 'butter'): 0.16513354, ('professor', 'doctor'): 0.1894201, ('student', 'professor'): 0.15959984, ('stock', 'egg'): -0.15497579, ('money', 'cash'): -0.187648, ('king', 'queen'): -0.15169734, ('bishop', 'rabbi'): 0.13654898, ('football', 'basketball'): 0.048974115, ('football', 'tennis'): -0.14065844, ('alcohol', 'chemistry'): 0.32783324, ('baby', 'mother'): -0.25939956, ('car', 'automobile'): 0.2816353, ('journey', 'voyage'): 0.11505744, ('coast', 'shore'): -0.01445962, ('brother', 'monk'): 0.1335751, ('journey', 'car'): 0.4015325, ('coast', 'hill'): 0.11684095, ('forest', 'graveyard'): -0.035572365, ('monk', 'slave'): -0.028782189, ('coast', 'forest'): -0.45452648, ('psychology', 'doctor'): 0.16211636, ('psychology', 'mind'): 0.07707339, ('psychology', 'health'): -0.22384164, ('psychology', 'science'): -0.22640274, ('planet', 'moon'): -0.07195163, ('planet', 'galaxy'): -0.019670986}


## 3. Comparison with the Gold Standard (1 mark)

### Question 9 (1.0 mark)

**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA', 'lm').

**Task**: Compute the Pearson correlation coefficient between the estimated similarity scores (Lin, NPMI and LSA similarities) and the gold standard similarity ratings. Produce *pearson_correlations*, a dictionary containing the methods as keys and correlations as values.

**Check**: Use the assertion statements in *"For your testing"* below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs! 

In [36]:
def sim_list(sim_dict):
    scores = []
    for i in sim_dict:
        scores.append(sim_dict[i])
    return scores

In [37]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA', 'lm'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###
lin_sim = sim_list(lin_similarities)
NPMI_sim = sim_list(NPMI_similarities)
LSA_sim = sim_list(LSA_similarities)
lm_sim = sim_list(lm_similarities)
gold_standard = sim_list(final_gold_standard)

pearson_correlations["lin"] = pearsonr(lin_sim, gold_standard)[0]
pearson_correlations["NPMI"] = pearsonr(NPMI_sim, gold_standard)[0]
pearson_correlations["LSA"] = pearsonr(LSA_sim, gold_standard)[0]
pearson_correlations["lm"] = pearsonr(lm_sim, gold_standard)[0]
###
# Your answer ENDS HERE
###

print(pearson_correlations)

{'lin': 0.5301489978447533, 'NPMI': 0.1879988886941005, 'LSA': 0.3874097848621173, 'lm': 0.1156140370994414}


<b>For your testing:</b>

In [38]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.